<h1><center> Topic Modeling: UN Speeches
    
<center> 
    Justine Huỳnh
 
<center> 2021/10/21

In [33]:
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair
from dfply import *
from plotnine import *
import seaborn as sns
import missingno
import random

# Stopwords from sklearn
# Stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw

#### Stopwords from nltk
import nltk
from nltk.corpus import stopwords
nltk_sw = stopwords.words("english")

#### Stopwrods from spacy
from spacy.lang.en.stop_words import STOP_WORDS as spacy_sw

## Sentiemtn Analysis
import textblob
import vaderSentiment

# Count vectorizer for word frequencies
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# Lemonizer
from nltk.stem.wordnet import WordNetLemmatizer
# Stemer
from nltk.stem import SnowballStemmer

# Topic modeling
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
from gensim import corpora
import pickle # saves the object into a file
from transformers import pipeline
# Images
from IPython.display import Image, HTML

import warnings
warnings.filterwarnings("ignore")

In [21]:
# def get_lemma(word, myLemonizer = WordNetLemmatizer()):
#     '''
#     Lemmatizes the word
    
#     @param String word the given word to lemmatize
#     @param myLemonizer the given lemmatizer; optional
#     @return the newly lemmatized core word
#     '''
    
# #     print("word lemonizer", WordNetLemmatizer().lemmatize(word))
#     # Should I do stemming too? Hmmmm
#     return myLemonizer.lemmatize(word)

# def get_stemmer(word, myStemmer = SnowballStemmer(language = "english")):
#     '''
#     Stems the word
    
#     @param String word the given word to stem
#     @param myStemmer the given stemmer; optional
#     @return the newly stemmed core word
#     '''
#     return myStemmer.stem(word)
    

# def removePunctuation(word):
#     '''
#     Removes punctuation from a given word
    
#     @param String word the given word
#     @return the word, without punctuation
#     '''
    
#     lastIndex = len(word) - 1
#     if word[lastIndex] in myPunc:
#         # eradicate the last index value
#         return word[:-1]

#     return word
    

# def prepare_text_for_lda(text, myLemonizer = WordNetLemmatizer(), myStemmer = SnowballStemmer(language = "english")):
#     '''
#     Tokenizes, splits, removes stopwords from the text.
#     Basically, preparing text for LDA (one type of topic model)
    
#     @param String text the given text
#     @param WordNetLemmatizer myLemonizer the given lemmatizer
#     @return the tokens
#     '''
#     text = text.lower()
    
#     # Split by spaces
#     tokens = text.split(" ")
    
#     # Stripping away newline characters and tabs
#     tokens = [word.strip("\n") for word in tokens]
#     tokens = [word.strip("\t") for word in tokens]
# #     tokens = [word.strip("\d") for word in tokens]
#     # Strip damn numbers
    
#     tokens = [token for token in tokens if len(token) > 1]
#     # a
    
#     tokens = [token for token in tokens if token not in both_sw]
# #     if "the" or "an" or "and" in tokens:
# # #         print(f"found the/an/and in: {tokens}")
# #         print("found the/an/and")
#     tokens = [get_lemma(token, myLemonizer) for token in tokens]
#     tokens = [get_stemmer(token, myStemmer) for token in tokens]
#     tokens = [removePunctuation(token) for token in tokens]
#     return tokens

# def getCleanedTokensList(data, myLemonizer = WordNetLemmatizer(), myStemmer = SnowballStemmer(language = "english")):
#     '''
#     Gets cleaned tokens. 
#     Processes include: lowercasing, splitting by spaces, strips spaces/tabs/newlines,
#     punctuation, lemmatizing, stems
    
#     @param DataFrame data the given data frame
#     @param WordNetLemmatizer myLemonizer the given lemmatizer
#     @param SnowballStemmer myStemmer the given stemmer
#     @return the list of tokens for each speech
#     '''
#     text_data = []

#     for line in data:
#         tokens = prepare_text_for_lda(line, myLemonizer, myStemmer)
#         text_data.append(tokens)
#     return text_data


# def generate_viz(data, NUM_TOPICS=5, myLemonizer = WordNetLemmatizer(), myStemmer = SnowballStemmer(language = "english")):
#     '''
#     Graphs the data by topic.
#     Also saves the files and model into other files
    
#     @param list data the given list of Strings
#     @param int NUM_TOPICS number of topics to graph; default set to 5
#     @param WordNetLemmatizer myLemonizer the given lemmatizer
#     @return the graph, dictionary, and corpus
#     '''
# #     text_data = []

# #     for line in data:
# #         tokens = prepare_text_for_lda(line, myLemonizer, myStemmer)
# #         text_data.append(tokens)
#     text_data = getCleanedTokensList(data, myLemonizer, myStemmer)

#     dictionary = corpora.Dictionary(text_data)
#     corpus = [dictionary.doc2bow(text) for text in text_data]
#     pickle.dump(corpus, open('corpus.pkl', 'wb'))
#     dictionary.save('dictionary.gensim')
#     ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15, random_state = 420)
#     ldamodel.save('model5.gensim')

#     topics = ldamodel.print_topics(num_words=4)
#     for topic in topics:
#         print(f"topic: {topic}")

#     dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
#     corpus = pickle.load(open('corpus.pkl', 'rb'))
#     lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

#     lda_display = gensimvis.prepare(lda, corpus, dictionary, sort_topics=False)
#     return lda_display, dictionary, corpus


In [34]:
HTML("Topic_Graphs_Country/China.html")

In [30]:
HTML("Topic_Graphs_Country/Philippines.html")

C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


### Steps

    1: Instantiate vectorizer with parameters: vec
    2: Vectorize column of text with fit_transform: X
    3: Instantiate LDA model with parameters: lda
    4: Fit LDA model to vectorized text: doc_topics

 jupyter nbconvert Presentation_Topic_Modeling.ipynb --to slides --no-prompt --TagRemovePreprocessor.remove_input_tags={\\"to_remove\\"} --post serve --SlidesExporter.reveal_theme=simple
 
 Note: don't forget the backslash to escape the quotation marks!